Засоби підготовки та аналізу даних

Лабораторна робота №2

Виконав: Радіонов Арсеній ФБ-22

Спочатку імпортуємо потрібні бібліотеки

In [12]:
import pandas as pd
import urllib.request
import os
from datetime import datetime, date
import csv

Тепер давайте створимо директорію та ініціалізуємо змінні для подальшої роботи

In [13]:
folder = 'lab_2'
web_link = ('https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={''}&year1=1981&year2=2024&type=Mean')
indices = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
if not os.path.exists(folder):
    os.makedirs(folder)

Тепер заванатжимо потрібні дані та перевіримо їх

In [17]:
for region_id, ind in indices.items():
    file_exists = False
    filename_start = 'NOAA_' + str(ind) + '_'
    for filename in os.listdir(folder):
        if filename.startswith(filename_start):
            print('file with index ' + str(ind) + ' already exists')
            file_exists = True
    if not file_exists:
        url_link = web_link.format(region_id)
        web_page = urllib.request.urlopen(url_link)
        page_text = web_page.read()
        current_time = datetime.now()
        date_time = current_time.strftime("%d-%m-%Y_%H-%M-%S")
        file_path = os.path.join(folder, 'NOAA_' + str(ind) + '_' + date_time + '.csv')
        with open(file_path, 'wb') as output_file:
            output_file.write(page_text)
        print('file ' + file_path + ' was created')
print('Success')

file with index 22 already exists
file with index 24 already exists
file with index 23 already exists
file with index 25 already exists
file with index 3 already exists
file with index 4 already exists
file with index 8 already exists
file with index 19 already exists
file with index 20 already exists
file with index 21 already exists
file with index 9 already exists
file with index 10 already exists
file with index 11 already exists
file with index 12 already exists
file with index 13 already exists
file with index 14 already exists
file with index 15 already exists
file with index 16 already exists
file with index 17 already exists
file with index 18 already exists
file with index 6 already exists
file with index 1 already exists
file with index 2 already exists
file with index 7 already exists
file with index 5 already exists
Success


Тепер з наших CSV файлів створимо датафрейм у нашій директорії. Виглядає таким чином:

In [18]:
region_names = {
    1: 'Вінницька', 2: 'Волинська', 3: 'Дніпропетровська', 4: 'Донецька', 5: 'Житомирська',
    6: 'Закарпатська', 7: 'Запорізька', 8: 'Івано-Франківська', 9: 'Київська',
    10: 'Кіровоградська', 11: 'Луганська', 12: 'Львівська', 13: 'Миколаївська', 14: 'Одеська',
    15: 'Полтавська', 16: 'Рівенська', 17: 'Сумська', 18: 'Тернопільська', 19: 'Харківська',
    20: 'Херсонська', 21: 'Хмельницька', 22: 'Черкаська', 23: 'Чернівецька', 24: 'Чернігівська',
    25: 'Республіка Крим'
}

def read_files_to_dataframe(directory, output_path):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'area']
    dataframe = pd.DataFrame(columns=headers)
    if os.path.exists(output_path):
        os.remove(output_path)
        print(f'File {output_path} already exists and has been removed.')
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            try:
                df = pd.read_csv(file_path, skiprows=2, names=headers)
                df['Year'] = df['Year'].str.replace('<tt><pre>', '').str.replace('</pre></tt>', '')
                region = int(filename.split('_')[1])
                df['area'] = region
                df['area'] = df['area'].replace({index: region for index, region in region_names.items()})
                df = df.drop(df.loc[df['VHI'] == -1].index)
                df = df.dropna()
                dataframe = pd.concat([dataframe, df], ignore_index=True)
                print(f'Successfully read file: {filename}')
            except pd.errors.ParserError:
                print(f'Error reading {filename}: ParserError')
    dataframe.to_csv(output_path, index=False)
    print(f'DataFrame saved to: {output_path}')
    return dataframe

df = read_files_to_dataframe('lab_2', 'all.csv')
print(df)

File all.csv already exists and has been removed.
Successfully read file: NOAA_10_24-03-2024_23-20-15.csv
Successfully read file: NOAA_11_24-03-2024_23-20-16.csv
Successfully read file: NOAA_12_24-03-2024_23-20-17.csv
Successfully read file: NOAA_13_24-03-2024_23-20-18.csv
Successfully read file: NOAA_14_24-03-2024_23-20-18.csv
Successfully read file: NOAA_15_24-03-2024_23-20-19.csv
Successfully read file: NOAA_16_24-03-2024_23-20-20.csv
Successfully read file: NOAA_17_24-03-2024_23-20-21.csv
Successfully read file: NOAA_18_24-03-2024_23-20-22.csv
Successfully read file: NOAA_19_24-03-2024_23-20-11.csv
Successfully read file: NOAA_1_24-03-2024_23-20-24.csv
Successfully read file: NOAA_20_24-03-2024_23-20-12.csv
Successfully read file: NOAA_21_24-03-2024_23-20-13.csv
Successfully read file: NOAA_22_24-03-2024_23-20-02.csv
Successfully read file: NOAA_23_24-03-2024_23-20-04.csv
Successfully read file: NOAA_24_24-03-2024_23-20-03.csv
Successfully read file: NOAA_25_24-03-2024_23-20-05.csv

C:\Users\arsen\AppData\Local\Temp\ipykernel_11332\2607098511.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, df], ignore_index=True)


Successfully read file: NOAA_5_24-03-2024_23-20-27.csv
Successfully read file: NOAA_6_24-03-2024_23-20-23.csv
Successfully read file: NOAA_7_24-03-2024_23-20-26.csv
Successfully read file: NOAA_8_24-03-2024_23-20-10.csv
Successfully read file: NOAA_9_24-03-2024_23-20-14.csv
DataFrame saved to: all.csv
       Year  Week    SMN     SMT    VCI    TCI    VHI            area
0      1982   1.0  0.045  261.12  36.68  41.79  39.23  Кіровоградська
1      1982   2.0  0.041  262.61  36.46  35.10  35.78  Кіровоградська
2      1982   3.0  0.041  263.82  36.49  30.79  33.64  Кіровоградська
3      1982   4.0  0.038  264.71  33.46  28.66  31.06  Кіровоградська
4      1982   5.0  0.034  264.76  29.66  30.50  30.08  Кіровоградська
...     ...   ...    ...     ...    ...    ...    ...             ...
53620  2024   7.0  0.105  268.78  61.68  29.21  45.41        Київська
53621  2024   8.0  0.123  272.20  66.04  22.41  44.19        Київська
53622  2024   9.0  0.141  275.18  69.86  18.22  44.01        Київсь

Робимо першу процедуру. Ряд "VHI" для області за рік вказаний нами.

In [19]:
def VHI_of_year_and_area(selected_year, selected_area):
    vhi_data = df[(df['Year'] == selected_year) & (df['area'] == selected_area)]
    if not vhi_data.empty:
        print(f"VHI data for the year {selected_year} and area {selected_area}:")
        vhi_values = ', '.join(map(str, vhi_data['VHI']))
        print(f"VHI values: {vhi_values}")
    else:
        print("No data available for the selected year and area.")

selected_year = input("Enter the year: ")
selected_area = input("Enter the area: ")
VHI_of_year_and_area(selected_year, selected_area)

VHI data for the year 1999 and area Київська:
VHI values: 37.7, 39.39, 43.82, 48.27, 52.55, 53.61, 52.3, 50.36, 47.0, 45.29, 44.57, 44.58, 45.43, 47.24, 51.1, 54.21, 58.33, 64.64, 67.36, 67.42, 64.85, 60.47, 56.36, 51.55, 47.28, 43.78, 43.3, 44.73, 46.39, 48.7, 50.45, 52.75, 53.88, 55.0, 56.07, 54.64, 50.68, 44.68, 41.87, 46.53, 51.19, 49.7, 47.31, 41.86, 38.46, 33.93, 29.4, 28.16, 26.79, 24.0, 21.31, 21.64


Шукаємо екстремуми (min та max), реалізовано наступним чином:

In [20]:
def VHI_of_minmax(selected_area, selected_year):
    found = df[(df["area"] == selected_area) & (df["Year"] == str(selected_year))]
    min_v = df[(df['Year'].astype(str) == str(selected_year)) & (df["area"] == selected_area)]['VHI'].min()
    max_v = df[(df['Year'].astype(str) == str(selected_year)) & (df["area"] == selected_area)]['VHI'].max()
    
    print(f"Minimum VHI value for {selected_area} in {selected_year} year: {min_v}")
    min_row = found[found['VHI'] == min_v]
    print("Rows with minimum VHI:")
    for index, row in min_row.iterrows():
        print(f"Week: {row['Week']}, VHI: {row['VHI']}")
    
    print(f"Maximum VHI value for {selected_area} in {selected_year} year: {max_v}")
    max_row = found[found['VHI'] == max_v]
    print("Rows with maximum VHI:")
    for index, row in max_row.iterrows():
        print(f"Week: {row['Week']}, VHI: {row['VHI']}")

selected_area = input('Enter your area: ')
selected_year = input('Enter your year: ')
VHI_of_minmax(selected_area, selected_year)

Minimum VHI value for 1999 in Київська year: nan
Rows with minimum VHI:
Maximum VHI value for 1999 in Київська year: nan
Rows with maximum VHI:


Робимо другу процедуру. За певний період часу вказані області мають свої значення індексу "VHI":

In [22]:
def VHI_for_several_years_and_areas(selected_areas, start_year, end_year):
    for year in range(start_year, end_year + 1):
        print(f"Year: {year}")
        filtered_data_year = df[df['Year'] == str(year)]
        for area in selected_areas:
            filtered_data_area = filtered_data_year[filtered_data_year['area'] == area]
            print(f"Area: {area}")
            if not filtered_data_area.empty:
                vhi_values = ', '.join(map(str, filtered_data_area['VHI']))
                print(f"VHI values: {vhi_values}")
            else:
                print("No data available for this area and year.")
        print()

selected_areas = ['Черкаська', 'Одеська']
start_year = 1999
end_year = 2001
VHI_for_several_years_and_areas(selected_areas, start_year, end_year)

Year: 1999
Area: Черкаська
VHI values: 38.74, 39.43, 43.12, 48.17, 52.85, 54.66, 54.68, 53.93, 52.66, 51.15, 48.28, 46.17, 46.93, 48.05, 50.86, 53.03, 57.89, 64.95, 67.39, 66.47, 63.66, 60.1, 57.28, 53.94, 50.54, 47.39, 45.97, 45.94, 46.13, 47.61, 49.44, 51.03, 50.84, 50.28, 50.31, 47.48, 41.5, 35.55, 32.9, 34.07, 39.53, 39.83, 38.5, 36.44, 35.71, 33.73, 30.99, 31.85, 34.14, 33.82, 32.14, 32.76
Area: Одеська
VHI values: 44.72, 45.74, 45.86, 45.3, 45.78, 47.31, 47.84, 47.96, 48.06, 48.19, 48.64, 49.8, 50.6, 52.66, 57.06, 62.63, 69.43, 74.03, 76.21, 75.8, 73.23, 71.36, 70.13, 68.45, 65.87, 63.56, 62.74, 60.9, 58.81, 57.51, 57.25, 57.07, 56.53, 57.68, 58.94, 57.5, 53.43, 49.69, 48.18, 49.11, 49.95, 45.23, 42.53, 40.45, 38.45, 35.88, 33.52, 33.16, 32.64, 29.71, 29.32, 30.54

Year: 2000
Area: Черкаська
VHI values: 35.79, 37.89, 37.46, 36.62, 37.63, 38.49, 36.49, 35.46, 36.99, 38.71, 36.82, 34.78, 33.67, 32.03, 32.38, 35.97, 43.15, 52.06, 57.4, 57.53, 56.7, 55.79, 56.79, 59.94, 62.76, 64.04,

Тепер потрібно визначити роки, коли екстремальні посухи вплинули на більшу кількість областей, ніж вказана межа в 20% (тобто 5 областей з 25).

In [23]:
def extreme_drought(percent_threshold=0.2):
    extreme_drought_years = []
    extreme_drought_areas = {}
    for year in df['Year'].unique():
        drought_areas = df[(df['Year'] == year) & (df['VHI'] < 15)]['area'].unique()
        drought_areas_count = len(drought_areas)
        if (drought_areas_count / 25) >= percent_threshold:
            extreme_drought_years.append(year)
            extreme_drought_areas[year] = drought_areas.tolist()
    return extreme_drought_years, extreme_drought_areas

percent_threshold = 0.05
extreme_drought_years, extreme_drought_areas = extreme_drought(percent_threshold)
print(f"Years with extreme droughts affecting more than {percent_threshold * 100}% of regions:")
for year in extreme_drought_years:
    min_vhi_value = df[(df['Year'].astype(str) == str(year))]['VHI'].min()
    print(f"Year: {year}, Affected areas: {extreme_drought_areas[year]}, Lowest VHI value: {min_vhi_value}")
    print()

Years with extreme droughts affecting more than 5.0% of regions:
Year: 1986, Affected areas: ['Луганська', 'Донецька'], Lowest VHI value: 12.45

Year: 1993, Affected areas: ['Луганська', 'Донецька'], Lowest VHI value: 6.26

Year: 2000, Affected areas: ['Харківська', 'Вінницька', 'Черкаська', 'Київська'], Lowest VHI value: 9.36

Year: 2007, Affected areas: ['Миколаївська', 'Одеська', 'Херсонська', 'Республіка Крим', 'Запорізька'], Lowest VHI value: 5.52



Реалізуємо для помірних посух

In [25]:
def moderate_drought():
    mod_dro_years = []
    mod_dro_regions = {}
    for year in df['Year'].unique():
        drought_regions = df[(df['Year'] == year) & (df['VHI'] > 15) & (df['VHI'] < 35)]['area'].unique()
        drought_regions_count = len(drought_regions)
        if (drought_regions_count / 25) >= 0.95:
            mod_dro_years.append(year)
            mod_dro_regions[year] = drought_regions.tolist()
    return mod_dro_years, mod_dro_regions

ext_dro_years, mod_dro_regions = moderate_drought()
print("Years with extreme droughts affecting more than 95% of regions:")
for year in ext_dro_years:
    print(f"Year: {year}, Affected areas: {mod_dro_regions[year]}")

Years with extreme droughts affecting more than 95% of regions:
Year: 1982, Affected areas: ['Кіровоградська', 'Луганська', 'Львівська', 'Миколаївська', 'Одеська', 'Полтавська', 'Рівенська', 'Сумська', 'Тернопільська', 'Харківська', 'Вінницька', 'Херсонська', 'Хмельницька', 'Черкаська', 'Чернівецька', 'Чернігівська', 'Республіка Крим', 'Волинська', 'Дніпропетровська', 'Донецька', 'Житомирська', 'Закарпатська', 'Запорізька', 'Івано-Франківська', 'Київська']
Year: 1983, Affected areas: ['Кіровоградська', 'Луганська', 'Львівська', 'Миколаївська', 'Одеська', 'Полтавська', 'Рівенська', 'Сумська', 'Тернопільська', 'Харківська', 'Вінницька', 'Херсонська', 'Хмельницька', 'Черкаська', 'Чернівецька', 'Чернігівська', 'Республіка Крим', 'Волинська', 'Дніпропетровська', 'Донецька', 'Житомирська', 'Закарпатська', 'Запорізька', 'Івано-Франківська', 'Київська']
Year: 1984, Affected areas: ['Кіровоградська', 'Луганська', 'Львівська', 'Миколаївська', 'Полтавська', 'Рівенська', 'Сумська', 'Тернопільська'